This will eventually be an introduction to numerical methods in partial differential equations. Over time, I hope to develop a resource which can be used to learn the theory and practice of numerical PDEs. The philosophy of this text is to be fully rigorous while connecting theory to practice with frequent coded examples of numerical solvers.

The essential prerequisites to be able to effectively use this text are calculus and some coding knowledge. The first portion of this text is about finite difference methods, which are in effect carefully chosen Taylor approximations. Most other things can be picked up while reading.

The theory of numerical PDEs cannot exist apart from PDEs. Numerical schemes which work well for one type of PDE may fail miserably when adapted to even the simplest PDE of a different type. More importantly, knowledge of how the PDE can be solved and its solution informs the design of a numerical method to approximate its solution. PDEs with known exact solutions can be good test beds for numerical methods.

Let's start with an overview of some common PDEs. I make an attempt to use $t$ to represent a time-like variable and $x$ to represent a space-like variable. If there are three dimensions in particular, may use $\mathbf = (x, y, z)$ I try to use a bold $\mathbf x$ to represent an $n$-dimensional space-like variable. The letters $u$ and $v$ commonly denote solutions to a PDE and are understood to be functions of $x$ and $t$. The partial derivative operators may be written as $\frac{\partial}{\partial{x}}$ or just $\partial_x$. The partial derivatives of $u$ are written as $u_x$ and $u_t$. Second derivatives are written similarly: $u_{tx} = (u_t)_x = \partial_x \partial t u$. We freely assume that all solutions are sufficiently smooth that mixed partial derivatives commute, i.e., that $u_{xt} = u_{tx}$, etc. Note that there are [well-known examples](https://en.wikipedia.org/wiki/Symmetry_of_second_derivatives) in which this is not true.

The heat equation in one dimension: $u_t + au_xx = 0$, where $a > 0$ is a positive constant. The function $u$ models the evolution of temperature in a one-dimensional medium of constant thermal diffusivity $a$.

The heat equation in three dimensions: $u_t + a(u_xx + u_yy + u_zz)$.

The wave equation: $u_tt = c^2(u_{xx} + u_{yy} + u_{zz})$.

The sum of all spacial second derivatives, $u_{xx} + u_{yy} + u_{zz}$, is common enough that it is given a dedicated symbol: $\Delta u$. So we can rewrite the heat equation as $u_t + a \Delta u = 0$ and the wave equation as $u_tt = c^2 \Delta u$. 
